In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkOlaEDA.com').getOrCreate()

In [2]:
spark

In [3]:
df=spark.read.csv('/content/drive/MyDrive/Abc/Ola_data/Bengaluru Ola.csv',header=True,inferSchema=True)
df.show(5)

+----------+-------------------+----------+-------------------+-----------+------------+---------------+-------------+--------+--------+----------------------+---------------------------------+-------------------------+-------------------------------+----------------+-----------------------+-------------+--------------+-------------+--------------+---------------+
|      Date|               Time|Booking ID|     Booking Status|Customer ID|Vehicle Type|Pickup Location|Drop Location|Avg VTAT|Avg CTAT|Cancelled  by Customer|Reason for Cancelling by Customer|Cancelled Rides by Driver|Reason for Cancelling by Driver|Incomplete Rides|Incomplete Rides Reason|Booking Value|Payment Method|Ride Distance|Driver Ratings|Customer Rating|
+----------+-------------------+----------+-------------------+-----------+------------+---------------+-------------+--------+--------+----------------------+---------------------------------+-------------------------+-------------------------------+---------------

In [4]:
df.dtypes

[('Date', 'string'),
 ('Time', 'timestamp'),
 ('Booking ID', 'string'),
 ('Booking Status', 'string'),
 ('Customer ID', 'int'),
 ('Vehicle Type', 'string'),
 ('Pickup Location', 'string'),
 ('Drop Location', 'string'),
 ('Avg VTAT', 'double'),
 ('Avg CTAT', 'double'),
 ('Cancelled  by Customer', 'int'),
 ('Reason for Cancelling by Customer', 'string'),
 ('Cancelled Rides by Driver', 'int'),
 ('Reason for Cancelling by Driver', 'string'),
 ('Incomplete Rides', 'int'),
 ('Incomplete Rides Reason', 'string'),
 ('Booking Value', 'double'),
 ('Payment Method', 'string'),
 ('Ride Distance', 'double'),
 ('Driver Ratings', 'double'),
 ('Customer Rating', 'double')]

In [5]:
from pyspark.sql.functions import *

In [6]:
df.groupBy('Booking Status').count().show()

+--------------------+-----+
|      Booking Status|count|
+--------------------+-----+
|             Success|33484|
| Cancelled by Driver| 9610|
|Cancelled by Cust...| 3799|
|          Incomplete| 3106|
+--------------------+-----+



In [7]:
df1=df.groupBy('Vehicle Type').agg(round(sum(col('Booking Value')),3).alias('Total Revenue')).sort(col('total revenue').desc())
df1.show()

+------------+-------------+
|Vehicle Type|Total Revenue|
+------------+-------------+
|        Bike|   4957790.69|
| Prime Sedan|   4919408.86|
|  Prime Plus|   4906974.66|
|       eBike|   4886694.02|
|   Prime SUV|   4884966.98|
|        Auto|   4870612.58|
|        Mini|    4840216.8|
+------------+-------------+



    All Successful rides

In [8]:
df_sucessful=df.where(col('Booking Status')=='Success')
df_sucessful.count()

33484

    Avg Distance of Each Vehcile Type

In [9]:
df_avg_dis=df.groupBy('Vehicle Type').agg(round(avg(col('Ride Distance')),3).alias('Average Distance')).sort(col('Average Distance').desc())
df_avg_dis.show()

+------------+----------------+
|Vehicle Type|Average Distance|
+------------+----------------+
|        Auto|           25.74|
|        Mini|          25.584|
| Prime Sedan|          25.509|
|        Bike|          25.411|
|   Prime SUV|          25.396|
|       eBike|          25.338|
|  Prime Plus|          25.154|
+------------+----------------+



    No of rides Cancelled by Customers

In [10]:
df_cancel=df.where(col('Cancelled  by Customer')==1)
df_cancel.count()

3799

    Top 5 Customers based on the total Booking value

In [11]:
df_top_cust=df.groupBy('Customer ID').agg(round(sum(col('Booking Value')),2).alias('Value')).sort(col('Value').desc())
df_top_cust.show(5)

+-----------+-------+
|Customer ID|  Value|
+-----------+-------+
|     580524| 5078.2|
|     599413|4263.67|
|     617275|3929.33|
|     428477|3909.54|
|     647438|3891.43|
+-----------+-------+
only showing top 5 rows



    Cancelled by Drivers with reason Personal & Car related issues

In [12]:
df_driver=df.where((col('Cancelled Rides by Driver')==1)&(col('Reason for Cancelling by Driver').like('%Personal%')))
df_driver.show(5)

+----------+-------------------+----------+-------------------+-----------+------------+---------------+-------------+--------+--------+----------------------+---------------------------------+-------------------------+-------------------------------+----------------+-----------------------+-------------+--------------+-------------+--------------+---------------+
|      Date|               Time|Booking ID|     Booking Status|Customer ID|Vehicle Type|Pickup Location|Drop Location|Avg VTAT|Avg CTAT|Cancelled  by Customer|Reason for Cancelling by Customer|Cancelled Rides by Driver|Reason for Cancelling by Driver|Incomplete Rides|Incomplete Rides Reason|Booking Value|Payment Method|Ride Distance|Driver Ratings|Customer Rating|
+----------+-------------------+----------+-------------------+-----------+------------+---------------+-------------+--------+--------+----------------------+---------------------------------+-------------------------+-------------------------------+---------------

In [13]:
df_driver.count()

2413

    Max and Min Ratings for Drivers

In [14]:
df_max_min=df.agg(max(col('Driver Ratings')).alias('Max_Rating'),min(col('Driver Ratings')).alias('Min_Rating'))
df_max_min.show()

+----------+----------+
|Max_Rating|Min_Rating|
+----------+----------+
|       5.0|       3.0|
+----------+----------+



    No of Rides with UPI as Payment mode

In [15]:
df_payment=df.where(col('Payment Method').like('%UPI%'))
df_payment.count()

8428

    Avg Rating for each vechile type where Booking status is success

In [16]:
df_avg_rating=df.where(col('Booking Status')=='Success').groupBy('Vehicle Type')\
                .agg(round(avg(col('Customer Rating')),3).alias('Average Rating'))\
                .sort(col('Average Rating').desc())
df_avg_rating.show()

+------------+--------------+
|Vehicle Type|Average Rating|
+------------+--------------+
|   Prime SUV|         4.013|
|  Prime Plus|         4.006|
|        Auto|         4.006|
|       eBike|           4.0|
| Prime Sedan|         3.999|
|        Bike|         3.996|
|        Mini|         3.992|
+------------+--------------+



    Sum of Revenue generated for each vechile type

In [17]:
df_total_booking=df.groupBy('Vehicle Type').agg(round(sum(col('Booking Value')),2).alias('Total Revenue')).sort(col('Total Revenue'))
df_total_booking.show()

+------------+-------------+
|Vehicle Type|Total Revenue|
+------------+-------------+
|        Mini|    4840216.8|
|        Auto|   4870612.58|
|   Prime SUV|   4884966.98|
|       eBike|   4886694.02|
|  Prime Plus|   4906974.66|
| Prime Sedan|   4919408.86|
|        Bike|   4957790.69|
+------------+-------------+



    Incomplete Rides,their Pickup Location and Reason

In [18]:
df_reason=df.where(col('Booking Status')=='Incomplete')
df_reason=df_reason.select('Vehicle Type','Pickup Location','Incomplete Rides Reason')

In [19]:
df_reason.show()

+------------+---------------+-----------------------+
|Vehicle Type|Pickup Location|Incomplete Rides Reason|
+------------+---------------+-----------------------+
|        Bike|        Area-41|            Other Issue|
|  Prime Plus|        Area-46|      Vehicle Breakdown|
|       eBike|        Area-41|      Vehicle Breakdown|
|        Mini|        Area-42|        Customer Demand|
|   Prime SUV|         Area-3|            Other Issue|
| Prime Sedan|        Area-15|      Vehicle Breakdown|
|       eBike|        Area-10|      Vehicle Breakdown|
|   Prime SUV|        Area-34|        Customer Demand|
|  Prime Plus|        Area-34|            Other Issue|
|   Prime SUV|        Area-29|        Customer Demand|
|   Prime SUV|        Area-12|        Customer Demand|
|        Auto|        Area-35|            Other Issue|
|        Mini|        Area-24|            Other Issue|
|        Mini|        Area-40|            Other Issue|
|  Prime Plus|        Area-48|      Vehicle Breakdown|
|       eB

**END OF CODE**